## Lab6-Assignment: Topic Classification

Use the same training, development, and test partitions of the the 20 newsgroups text dataset as in Lab6.4-Topic-classification-BERT.ipynb 

* Fine-tune and examine the performance of another transformer-based pretrained language models, e.g., RoBERTa, XLNet

* Compare the performance of this model to the results achieved in Lab6.4-Topic-classification-BERT.ipynb and to a conventional machine learning approach (e.g., SVM, Naive Bayes) using bag-of-words or other engineered features of your choice. 
Describe the differences in performance in terms of Precision, Recall, and F1-score evaluation metrics.

### 1. Installs & Imports

In [ ]:
!pip install simpletransformers --upgrade

In [1]:
import pandas as pd
import torch
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from simpletransformers.classification import ClassificationArgs, ClassificationModel


### 2. Dataset Loading & Splitting

In [2]:
# Load only the four specific categories we need
categories = ["alt.atheism", "comp.graphics", "sci.med", "sci.space"]

# Strip out headers, footers, and quoted text to prevent overfitting
train_groups = fetch_20newsgroups(
    subset="train",
    remove=("headers", "footers", "quotes"),
    categories=categories,
    random_state=42,
)
test_groups = fetch_20newsgroups(
    subset="test",
    remove=("headers", "footers", "quotes"),
    categories=categories,
    random_state=42,
)

In [3]:
# Check if identical to notebook 6.4
print("Distribution in Training Set:")
print(dict(sorted(Counter(train_groups.target).items(), key=lambda x: x[0])))
print("\nDistribution in Test Set:")
print(dict(sorted(Counter(test_groups.target).items(), key=lambda x: x[0])))

Distribution in Training Set:
{0: 480, 1: 584, 2: 594, 3: 593}

Distribution in Test Set:
{0: 319, 1: 389, 2: 396, 3: 394}


In [4]:
# Convert the training and test sets to dataframes
train_df = pd.DataFrame({"text": train_groups.data, "labels": train_groups.target})
test_df = pd.DataFrame({"text": test_groups.data, "labels": test_groups.target})

# Split the training set into two, so that 10% of it can be used as validation set
train_df, dev_df = train_test_split(
    train_df, test_size=0.1, random_state=0, stratify=train_df[["labels"]]
)

In [5]:
# Check if identical to notebook 6.4
print("Distribution in Training Set:")
print(dict(sorted(Counter(train_df["labels"]).items(), key=lambda x: x[0])))
print("\nDistribution in Validation Set:")
print(dict(sorted(Counter(dev_df["labels"]).items(), key=lambda x: x[0])))

Distribution in Training Set:
{0: 432, 1: 525, 2: 534, 3: 534}

Distribution in Validation Set:
{0: 48, 1: 59, 2: 60, 3: 59}


### 3. Finetune RoBERTa for Topic Classification on the Dataset

In [6]:
# Model Configuration
model_args = ClassificationArgs()

# Overwrite existing saved models in the same directory
model_args.overwrite_output_dir = True  

# Enable evaluation during training to monitor performance
model_args.evaluate_during_training = True  

# Training parameters
model_args.num_train_epochs = 10  # Train for 10 epochs
model_args.train_batch_size = 32  # Process 32 samples per batch
model_args.learning_rate = 4e-6  # Learning rate for optimization
model_args.max_seq_length = 256  # Max token length per input (the higher the number, the longer it takes)

# Early stopping helps prevent overfitting by stopping training 
# when validation loss stops improving
model_args.use_early_stopping = True
model_args.early_stopping_delta = 0.01  # Minimum improvement in loss required to continue training
model_args.early_stopping_metric = "eval_loss"  # The metric to monitor
model_args.early_stopping_metric_minimize = True  # Lower eval_loss is better
model_args.early_stopping_patience = 2  # Stop training if no improvement in 2 evaluations

# Run validation every 32 training steps to track progress
model_args.evaluate_during_training_steps = 32

In [7]:
model = ClassificationModel(
    model_type = "roberta",
    model_name = "roberta-large",
    num_labels = 4,
    args = model_args,
    use_cuda = torch.cuda.is_available(), 
)

print("\n".join(str(model.args).split(",")))

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
training_results = model.train_model(train_df, eval_df = dev_df) # this fine tuning takes a lot of time, pls run it and lmk if it works

history = training_results[1]

  0%|          | 0/4 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true 

In [ ]:
# Training and evaluation loss
train_loss = history['train_loss']
eval_loss = history['eval_loss']
plt.plot(train_loss, label='Training loss')
plt.plot(eval_loss, label='Evaluation loss')
plt.title('Training and evaluation loss')
plt.xlabel('Training steps')  
plt.ylabel('Loss')
plt.legend()

In [ ]:
# Evaluate the model 
predicted, probabilities = model.predict(test_df.text.to_list())
test_df_copy = test_df.copy()
test_df_copy['predicted'] = predicted
test_df_copy.head(10)

In [ ]:
# Generate classification report
print(classification_report(test_df_copy['labels'], test_df_copy['predicted']))

#### *Comparison of Fine-tuned RoBERTa to Fine-Tuned BERT*

![BERT classification report](./BERT_classification_report.png)

`to be added`

### 4. Conventional ML Approach: `specify`

#### SVM with BoW and TF-IDF

In [9]:
from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

In [10]:
# Merging train and validation sets from above as validation set isn't needed with SVM/NB
X_train = pd.concat([train_df["text"], dev_df["text"]])
X_test = test_df["text"]
y_train = pd.concat([train_df["labels"], dev_df["labels"]])
y_test = test_df["labels"]

In [11]:
# BoW
vectorizer = CountVectorizer()
X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)

# Initialize and train the SVM
svm_model = svm.LinearSVC()
svm_model.fit(X_train_bow, y_train)

# Evaluate the model
y_pred = svm_model.predict(X_test_bow)
print(classification_report(y_test, y_pred, target_names=categories))

               precision    recall  f1-score   support

  alt.atheism       0.74      0.70      0.72       319
comp.graphics       0.72      0.84      0.78       389
      sci.med       0.80      0.69      0.74       396
    sci.space       0.71      0.73      0.72       394

     accuracy                           0.74      1498
    macro avg       0.74      0.74      0.74      1498
 weighted avg       0.74      0.74      0.74      1498



/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [17]:
# TF-IDF
vectorizer = TfidfVectorizer()
# vectorizer = TfidfVectorizer(min_df=2) -> Slightly better performance
X_train_tf = vectorizer.fit_transform(X_train)
X_test_tf = vectorizer.transform(X_test)

# Initialize and train the SVM
svm_model = svm.LinearSVC()
svm_model.fit(X_train_tf, y_train)

# Evaluate the model
y_pred = svm_model.predict(X_test_tf)
print(classification_report(y_test, y_pred, target_names=categories))

               precision    recall  f1-score   support

  alt.atheism       0.84      0.77      0.80       319
comp.graphics       0.88      0.88      0.88       389
      sci.med       0.88      0.83      0.85       396
    sci.space       0.77      0.86      0.82       394

     accuracy                           0.84      1498
    macro avg       0.84      0.84      0.84      1498
 weighted avg       0.84      0.84      0.84      1498



#### NB with BoW and TF-IDF

In [13]:
# BoW
vectorizer = CountVectorizer()
X_train_bow_nb = vectorizer.fit_transform(X_train)
X_test_bow_nb = vectorizer.transform(X_test)

# Initialize and train the Naive Bayes
nb_model = MultinomialNB()
nb_model.fit(X_train_bow_nb, y_train)

# Evaluate the model
y_pred = nb_model.predict(X_test_bow_nb)
print(classification_report(y_test, y_pred, target_names=categories))

               precision    recall  f1-score   support

  alt.atheism       0.78      0.89      0.83       319
comp.graphics       0.91      0.88      0.90       389
      sci.med       0.84      0.90      0.87       396
    sci.space       0.91      0.77      0.84       394

     accuracy                           0.86      1498
    macro avg       0.86      0.86      0.86      1498
 weighted avg       0.87      0.86      0.86      1498



In [18]:
# TF-IDF
vectorizer = TfidfVectorizer()
# vectorizer = TfidfVectorizer(min_df=2) -> Slightly better performance
X_train_tf_nb = vectorizer.fit_transform(X_train)
X_test_tf_nb = vectorizer.transform(X_test)

# Initialize and train the Naive Bayes
nb_model = MultinomialNB()
nb_model.fit(X_train_tf_nb, y_train)

# Evaluate the model
y_pred = nb_model.predict(X_test_tf_nb)
print(classification_report(y_test, y_pred, target_names=categories))

               precision    recall  f1-score   support

  alt.atheism       0.88      0.71      0.79       319
comp.graphics       0.90      0.87      0.88       389
      sci.med       0.76      0.91      0.83       396
    sci.space       0.83      0.82      0.83       394

     accuracy                           0.83      1498
    macro avg       0.84      0.83      0.83      1498
 weighted avg       0.84      0.83      0.83      1498



#### *Comparison of Fine-Tuned RoBERTa to Conventional ML Approach `specify`*

`to be added`